In [2]:
from IPython.display import display
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
from qutip import Bloch, Qobj
import time
import io

In [ ]:
%matplotlib agg

In [3]:
#X = np.array([[0.,1.],[1.,0.]])
#Z = np.array([[1.,0.],[0.,-1.]])

#def rabi_factory(detuning, omega_rabi):
#    H = -(detuning/2)*Z + omega_rabi/2*X
#    v,w = np.linalg.eigh(H)
#    psi0 = w[0,:].conj()
#    rabi = sqrt(detuning**2 + omega_ra@bi**2)
#    Npts = ceil(50/rabi).astype(int)
#    t = linspace(0,2*pi/rabi,Npts)
#    states = [ Qobj( w @ diag(exp(-1j*v*t_)) @ psi0) for t_ in t[:-1]]
#    return states

def rabi_factory(detuning, omega_rabi):
    rabi = np.sqrt(detuning**2 + omega_rabi**2)
    Npts = np.ceil(50/rabi).astype(int)
    t = np.linspace(0,2*np.pi/rabi,Npts)
    C = np.cos(rabi*t/2)
    S = np.sin(rabi*t/2)/rabi
    c1 = (C+1j*detuning*S)
    c2 = -1j*omega_rabi*S
    states = [ Qobj([c1[i],c2[i]]) for i in range(Npts-1)]
    return states

In [6]:
class ProgressPlot:
    """
    Real time plot to monitor a QM job
    """
    def __init__(self):
        self.states = None
        with plt.ioff():
            self.fig = plt.figure()
        self.bloch = Bloch(fig=self.fig)
        self.image = widgets.Image(value=self.make_plot(),format='png')
        self.slider_amplitude = widgets.FloatSlider(value=1.0,min=0.5,max=2.0,step=0.1,description='Amplitude:')
        self.slider_detuning = widgets.IntSlider(value=0,min=-10,max=10,description='Detuning:')
        self.slider_amplitude.observe(self.on_amplitude_change, names='value')
        self.slider_detuning.observe(self.on_detuning_change, names='value')
        self.states = rabi_factory(self.slider_detuning.value, self.slider_amplitude.value)
        self.player = widgets.Play(min=0, max=len(self.states)-1, step=1, interval=200, repeat=True)
        self.player.observe(self.play_frame, names='value')
        self.show()

    def make_plot(self):        
        if self.states:
            self.bloch.clear()
            self.bloch.add_states(self.states,'point',colors='C0')
            self.bloch.point_marker='.'
            self.bloch.add_states(self.states[self.player.value % len(self.states)],'vector',colors='C1')
        self.bloch.render()
        s = io.BytesIO()
        self.fig.savefig(s, format="png",bbox_inches='tight', pad_inches=0.5)
        return s.getbuffer()

    def show(self):
        display(self.image,
                widgets.HBox([self.player,self.slider_amplitude,self.slider_detuning,widgets.Label('MHz')]),)
        
    def on_amplitude_change(self,change):
        self.states = rabi_factory(self.slider_detuning.value, self.slider_amplitude.value)
        self.player.value = 0
        self.player.max = len(self.states)-1

    def on_detuning_change(self,change):
        self.states = rabi_factory(self.slider_detuning.value, self.slider_amplitude.value)
        self.player.value = 0
        self.player.max = len(self.states)-1
        
    def play_frame(self,change):
        self.image.value = self.make_plot()
                
pp=ProgressPlot();

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02D\x00\x00\x02D\x08\x06\x00\x00\x00\xf7\xe0;Z\x00\…